In [1]:
%pwd

'/workspaces/Credit_Card_Behaviour_Predicition/notebook'

In [2]:
import os
os.chdir('../')
%pwd

'/workspaces/Credit_Card_Behaviour_Predicition'

In [3]:
import pandas as pd

data=pd.read_csv("artifacts/data_ingestion/raw/1xo0d7jACK7yxmMpGA-PnPLLRxv5aLuxI.csv")
data.head()

,account_number,bad_flag,onus_attribute_1,transaction_attribute_1,transaction_attribute_2,transaction_attribute_3,transaction_attribute_4,transaction_attribute_5,transaction_attribute_6,transaction_attribute_7,...,bureau_enquiry_47,bureau_enquiry_48,bureau_enquiry_49,bureau_enquiry_50,onus_attribute_43,onus_attribute_44,onus_attribute_45,onus_attribute_46,onus_attribute_47,onus_attribute_48
0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,221000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,25000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,86000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,215000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(data, title="Profiling Report")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE: str
    csv_dir:Path
    all_schema:dict

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            csv_dir = config.csv_dir,
            all_schema=schema
        )

        return data_validation_config

In [8]:
import os
from src import logger

In [9]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True
            csv_files = [f for f in os.listdir(self.config.csv_dir) if f.endswith('.csv')]

            if not csv_files:
                raise ValueError("No CSV files found in the provided directory")

            for csv_file in csv_files:
                csv_path = os.path.join(self.config.csv_dir, csv_file)

                data = pd.read_csv(csv_path)
                all_cols = list(data.columns)

                all_schema = self.config.all_schema.keys()

                for col in all_cols:
                    if col not in all_schema:
                        validation_status = False
                        print(f"Column '{col}' in file '{csv_file}' is not in the schema.")

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            print(f"Error occurred: {e}")
            raise e

In [11]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-01-12 05:55:24,648: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-12 05:55:24,649: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-12 05:55:24,721: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-12 05:55:24,724: INFO: common: created directory at: artifacts]
[2025-01-12 05:55:24,725: INFO: common: created directory at: artifacts/data_validation]
